# Motor control performance analysis - Template notebook

<span style='color:red;font-style: italic;font-family:monospace'>This is a template document which must be modified. Some red text is written on various locations in this document to help the writer to do so.</span>

## Introduction

### Context and objective

This notebook aims at analysing data about motor control. It is tried to assess every usual performance indicator of a motor control. Raw data comes from previous measurements done separatly.

<span style='color:red;font-style: italic;font-family:monospace'>Complete this by precising where data comes from, what are the samples with their versions, and what the results below will be used for.</span>

### How to read this document

This document is a [Jupyter notebook](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html). Here is a definition of it extracted from the official documentation:

> Notebook documents [...] are documents produced by the Jupyter Notebook App, which contain both computer code (e.g. python) and rich text elements (paragraph, equations, figures, links, etc…). Notebook documents are both human-readable documents containing the analysis description and the results (figures, tables, etc..) as well as executable documents which can be run to perform data analysis.

The following table explains the purpose of each part of this document. As this document is very technical, **do no hesitate to jump directly to the part of interest for your needs.**

<span style='color:red;font-style: italic;font-family:monospace'>Complete this table with your additional parts.</span>

<table>
<thead>
<tr>
<th>Section</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td><a href="#About-data">About data</a></td>
<td>Details the content of data used to compute performance indicators.</td>
</tr>
<tr>
<td><a href="#About-analysis">About analysis</a></td>
<td>Details how data is sorted, what are the indicators and how they are calculated.</td>
</tr>
<tr>
<td><a href="#Import-of-Python-packages">Import of Python packages</a></td>
<td>Pure python code, to import packages used in the rest of the notebook.</td>
</tr>
<tr>
<td><a href="#Declaration-of-constants">Declaration of constants</a></td>
<td>Python code with explanations to declare some constants used in the rest of the notebook.</td>
</tr>
<tr>
<td><a href="#Import-of-raw-data">Import of raw data</a></td>
<td>Details about the source of data and Python code to import them.</td>
</tr>
<tr>
<td><a href="#Useful-generic-functions">Useful generic functions</a></td>
<td>Declaration of Python functions that are required to browse data, do the analysis and display charts.</td>
</tr>
<tr>
<td><a href="#Analysis">Analysis</a></td>
<td>Compute performance indicators and display synthesis pages.</td>
</tr>
</tbody>
</table>


## About data

### About the protocols

Two kinds of protocols have been applied to collect data.

* Apply a series of step commands on the input position of the actuator on the whole available angular range. Different amplitudes and slope of steps are used successively. <img src="resources/step_protocol.png" alt="Step illustration" width="600">
* Apply a sine curve on the input position of the actuator. The sine is centered on a chosen starting angle (see postures below). Different amplitudes and frequencies are used successively. <img src="resources/sinus_protocol.png" alt="Step illustration" width="600">

<span style='color:red;font-style: italic;font-family:monospace'>Complete this by precising how data has been retrieved, what is the sample rate, and where they are stored.</span>

### About the file structures

Here are the different types of files this notebook is dealing with:

* **Timeseries of sine movements**  
  CSV files (`.csv`) with a column `Time` and some columns for actuators and sensors data.
  Their file name must *contain* `<system_name>_sine` where `<system_name>`
  is the name of the system which is tested in this file (it can be a joint name, a motor name, a module name, etc).  
  Example: `Joint1_sine_measurement1.csv`  
  *WARNING:* TIME MUST BE IN SECONDS. It can be a float to indicate milliseconds.
* **Timeseries of step movements**  
  CSV files with a column `Time` and some columns for actuators and sensors data.
  Their file name must *contain* `<system_name>_step` where `<system_name>`
  is the name of the system which is tested in this file (it can be a joint name, a motor name, a module name, etc).  
  Example: `Joint1_step_measurement1.csv`
* **Metadata of sine movements**  
  YAML files (`.yaml`) which contain metadata about a timeseries file for sine movements.
  Their file name must be the same than the CSV file, except the `.yaml` extension.  
  Example: `Joint1_sine_measurement1.yaml`  
  They must contain a mapping named `MeasurementInfo` with information about the system and its state,
  with at least the key `SystemName` being the name of the system under study.
  For instance, if the system under study is a joint on a robot, there must be the following entries:
    * `RobotID`
    * `StartPosition`
    * `SystemName` (in this case, it is the name of the joint)
* **Metadata of step movements**  
  Same description than for sine movements, but for steps.


### About data source

To learn more about data which is downloaded and processed in this study, please refer to [the section about data import](#Import-of-raw-data).

## About analysis

### Performance indicators

#### Indicators based on a step command

Extracted from [Wikipedia](https://en.wikipedia.org/wiki/Step_response):

> The step response of a system in a given initial state consists of the time evolution of its outputs when its control input change from zero to one in a very short time. Formally, knowing the step response of a dynamical system gives information on the stability of such a system, and on its ability to reach one stationary state when starting from another. 

The figure below presents the characteristics (performance indicators) which can be extracted from a response to a step, supposing that the system is a *second order system* (which is an approximation for the joints of the robot). The meaning of each indicator is detailed afterwards.

![](resources/step_kpis.png)

* **Noise band**  
  A range of percentage in which the response stay after stabilizing. It has been decided to choose 10% after a first analysis of data.
* **$t_{10}$: 10% Rising time**  
  The time taken by the system to reach 10% of the response amplitude. If $y$ is the response signal, $y(t_{10}) = 0.1 \times \big(y(\infty) - y(0)\big)$
* **$t_{90}$: 90% Rising time**  
  The time taken by the system to reach 90% of the response amplitude. If $y$ is the response signal, $y(t_{90}) = 0.9 \times \big(y(\infty) - y(0)\big)$
* **$t_o$: Overshoot time**  
  The time taken by the system to reach the peak of the first overshoot, if there is an overshoot.
* **$t_s$: Settling time**  
  The time taken by the system to reach and stay in its *noise band*. This definition is specific for the needs of this study and can be different from what can be found in scientific literature. In our case, with a noise band of ±5%, it is the lowest $t_s$ so that $|y(t) - y(\infty)| \leq 0.05 y(\infty), \enspace \forall t \geq t_s$
* **$o$: Overshoot**  
  If $y$ is the response signal, it is defined by $O = \frac{y(t_o) - y(\infty)}{y(\infty) - y(0)}$. </br>
* **$e$: Steady state error**  
  The difference between the final value of the response and the step value. When the response is noisy, the final value of the response is the mean of the noise band.

Another indicator which is calculated but not visible on the position response is the **maximum current** reached during the movement.

#### Indicators based on a sinusoidal command

The sine response of a system in a given initial state consists of the time evolution of its outputs when its control input is a sine wave. It allows to get the frequency response of the system.
Extracted from [Wikipedia](https://en.wikipedia.org/wiki/Frequency_response):

> Frequency response is the quantitative measure of the output spectrum of a system in response to a stimulus, and is used to characterize the dynamics of the system. It is a measure of magnitude and phase of the output as a function of frequency, in comparison to the input. In simplest terms, if a sine wave is injected into a system at a given frequency, a linear system will respond at that same frequency with a certain magnitude and a certain phase angle relative to the input.

The figure below presents the characteristics (performance indicators) which can be extracted from a response to a sine wave. The meaning of each indicator is detailed afterwards.

![](resources/sine_kpis.png)

* **$Δ_φ$: Phase shift**  
  The difference of phase between the input and the response. It can be given as a time delay (knowing the frequency), or as an angle (radians or degrees).
* **$G$: Gain**  
  The ratio of the response amplitude over the input amplitude.
* **$of$: Offset**  
  The difference of mean value between the response and the input.


### More about response times

* Steps cannot really be discontinuous nor have a slope near from infinity because tests are done on real actuators. A maximum speed is not exceeded to avoid breaking the joints.
* Moreover, amplitude of steps is not a unit, because it would restrict the operating points of the joints. Several different amplitudes are used for the steps to get various data.

Because of the two points above, an absolute rising time of the control cannot be directly measured on data as it changes in function of the slope and the amplitude of the step. What can be done is to plot the response time in function of the amplitude and slope. To avoid having two parameters (amplitude and slope), amplitude is normalized to 1 and the normalized speed only will be used in the chart. It is therefore obtained the curve of the *rising time* ($T_r$) in function of *normalized speed of the unit step* ($S_s$). This curve can be interpolated by the following equation: 
\begin{equation}
\forall \{a,b,S_s\} \in \Re^+ :\quad  T_r(S_s) = \frac{a}{S_s} + b
\end{equation}
This equation presents two asymptotes. **The horizontal asymptote corresponds to the rising time of the system for a step with an infinite speed**. It is the value of this horizontal asymptote which is used to charaterize the control. But to perform comparison on real systems, the whole curve is necessary as infinite speed cannot be reached.

In the same way, an asymptote can be found for the maximum *rising speed* ($V_r$) of the system. The difference is the equation of the curve which is, in this case:
\begin{equation}
\forall \{a,b,S_s\} \in \Re^+ :\quad  V_r(S_s) = a \times (1 - \exp(-\frac{S_s}{b}))
\end{equation}

### Operating points

The parameters used to make the difference between operating points are:
* **Steady state current**  
  Image of the torque.
* **Load direction**  
  Whether the motor tries to counter the load (the gravity) or is dragged by it.
  It can also be defined as the direction of the torque compared to the direction of the movement.
  This parameter only concerns the steps and not the sine waves, as in a sinusoidal movement the
  joint is alternatively following and countering the gravity.
* **Temperature**  
  Temperature can change the constants of the motor.

Some groups are made using these three parameters, and each one of these groups is an operating point (or operating group) which has its own performances. System under studies are approximated as **second order linear systems** inside the ranges of an operating group.

To be able to compute performance indicators with enough confidence, data size must be large enough. It has been decided not to keep operating groups with less than **50 samples**.

### Representations of indicators

There are two types of charts which are used to display values of indicators about steps.

* One is a simple box plot:
  - Lower line of the box is the 1st quartile
  - Upper line of the box is the 3rd quartile
  - Middle line of the box is the median
  - Upper and lower whiskers are the limits before outliers, and are defined as $median \pm 1.5 \times (quartile_3 - quartile_1)$
* The other is a scatter plot of the indicator value in function of the speed of the step.
  It is not the actual speed of the step, but its speed once the amplitude of the step has been normalized to be equal to 1.
  Therefore, if $A$ is the actual amplitude of the step and $S$ its actual speed, then its normalized amplitude is $\frac{A}{A}$
  and its normalized speed is $\frac{S}{A}$.   
  Some orange lines are visible on this chart, they must be understood like a box plot, but with moving limits in function of
  the step speed.

![types of charts](resources/types_of_charts.png)

### Synthesis pages

For each joint and for each operating group, two synthesis pages are generated: 
  * one presenting the performance indicators based on a step,
  * and the other presenting the performance indicators based on a sine wave.

Each page contains information about the operating group, an overview of the raw responses, and values of performance indicators. Performance indicators do not have a single value, statistical results are given for each one of them instead, based on all the available data of the operating group. This allows to create a kind of abacus for the response of a joint.

Here is an example of a sythesis page:

<img src="resources/step_synthesis_example.png" alt="Synthesis example" width="600">

## Import of Python packages

### Built-in packages

In [2]:
# Add necessary built-in packages here

### Installed packages

In [3]:
# Add necessary installed packages here

### Local packages

In [3]:
from utils import signal as sigutils
from utils import data as datautils
from utils import plot as plotutils

## Declaration of constants

<span style='color:red;font-style: italic;font-family:monospace'>
    Add here any Python constant that may be useful in the rest of the notebook.
    It is also better if you explain the purpose and the content of these constants
    in markdown.
</span>

## Import of raw data

<span style='color:red;font-style: italic;font-family:monospace'>
    Explain here where your data comes from and how you collected it.
    Write code to fetch files or re-organize data if necessary. For instance if your files are on a FTP server, you can download them in this part of the notebook. If your CSV files need some cleaning before being processed, you can also do it here.
</span>

## Useful generic functions

All of the functions that can be used in several notebooks or scripts are written in seperate files in the module `utils`. Functions that are written in this notebook in the following section are specific for the goals of this notebook and it is not interesting to write them in some side files. 

To have a deep understanding of the data processing and chart plotting, the reader is invited to read the content of the `utils` package. It contains functions to:
  * process timeseries, 
  * compute performance indicators from step and sine signals,
  * create custom charts
  * create custom images

### Data browsing and reading

Some of the functions that are used to fetch data are in a separate file `utils.data`. Here are explained the most important functions:
  * `get_all_processed_data`: Get all the processing results for steps or sines from the given CSV files.
  * `get_operating_groups`: Split results into groups, one for each operating point.
  * `store_op_data`: Store data of an operating point in a pickle file.
  * `load_op_data`: Load data of an operating point from a pickle file.
  
<span style='color:red;font-style: italic;font-family:monospace'>Add here the custom functions you need to get and browse your data.</span>

In [1]:
def get_step_csv_paths():
    # PLEASE COMPLETE THIS FUNCTION AS IT IS USED IN THE REST OF THE TEMPLATE
    return

### Signal processing

All of the functions that are used to process timeseries are in separate files in the package `utils.signal`. Here are explained the most important functions:
  * `process_sine_signal`: processes a .csv file which contains the logs of a sinusoidal movement. It extracts the `pandas` DataFrame from it and the motor control performance indicators that can be found on a sinusoidal movement.
  * `process_steps_signal` processes a .csv file which contains the logs of a sequence of steps. It detects and extract each step from the whole signal. For each step, it returns the `pandas` DataFrame containing it and the motor control performances indicators that can be found on a single step.
  * `get_normalized_position_timeseries`: used to be able to compare several signals of different offsets and amplitudes.
  * `get_data_fit`: Used to fit a theorical curve equation on real data.

### Display

All of the functions that are used to process timeseries are in separate files in the package `utils.signal`. Here are explained the most important functions:

* `display_step_responses`: Display all normalized step commands and responses on a single chart.
* `display_bode`: Display the Bode diagram based en processed sine waves.
* `display_kpi_over_step_speed`: Display a chart of a KPI in function of normalized step speed.
* `display_kpi_distribution`: Get and display the distribution of a KPI for given processing results.
* `display_op_groups`: Display the data distribution among operating groups.
* `display_step_synthesis_pages`: Build, display and save KPI synthesis for steps.
* `display_sine_synthesis_pages`: Build, display and save KPI synthesis for sine waves.


## Analysis

### (To be replaced:) System name

In [1]:
system_name = 'system_name'

#### Data collection

First, process data and extract every step and sine and its associated KPIs from all the raw data files.

In [ ]:
csv_paths = get_step_csv_paths() # THIS FUNCTION IS EMPTY IN THE TEMPLATE, PLEASE WRITE IT
processed_steps = datautils.get_all_processed_data(csv_paths)
data_size = len(processed_steps)
print('After processing all the available data, a total of {} steps '
      'have been collected on {}.'.format(data_size, system_name))

In [ ]:
csv_paths = get_sine_csv_paths() # THIS FUNCTION IS EMPTY IN THE TEMPLATE, PLEASE WRITE IT
processed_sines = datautils.get_all_processed_data(csv_paths)
data_size = len(processed_sines)
print('After processing all the available data, a total of {} steps '
      'have been collected on {}.'.format(data_size, system_name))

#### Operating points split

Steps are split into several groups, each group being an operating point of the motor. To learn more about how operating points are selected, please refer to [section about operating points](#Operating-points).

In [ ]:
# For this joint, it is decided to split
# current consumptions into ranges of 0.5A
current_bin_size = 0.5

# and temperature into ranges of 30°C
temperature_bin_size = 30

In [ ]:
step_op_groups, uncat = datautils.get_operating_groups(processed_steps, current_bin_size,
                                      temperature_bin_size, min_size=50)

print(('{} operating groups are obtained for steps\n'
      '{} steps are uncategorizable.').format(len(step_op_groups), len(uncat)))

plotutils.display_op_groups(step_op_groups, data_type='step')

In [ ]:
sine_op_groups, uncat = datautils.get_operating_groups(processed_sines, current_bin_size, 
                                      temperature_bin_size, min_size=1)

print(('{} operating groups are obtained for sines\n'
      '{} sines are uncategorizable.').format(len(sine_op_groups), len(uncat)))

plotutils.display_op_groups(sine_op_groups, data_type='sine')

#### Synthesis

For each operating point, a page of synthesis is generated. To learn more about them, please refer to section ["section about synthesis pages"](#Synthesis-pages)

In [ ]:
imgs = plotutils.display_step_synthesis_pages(processed_steps, step_op_groups, system_name)

In [ ]:
imgs = plotutils.display_sine_synthesis_pages(processed_sines, sine_op_groups, system_name)